In [1]:
import requests
from bs4 import BeautifulSoup

def scrape_weather_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 気温のデータを取得
    temperature_data = []
    for row in soup.find_all('tr', class_='mtx'):
        columns = row.find_all('td')
        if len(columns) >= 3:
            date = columns[0].text.strip()
            temperature = columns[2].text.strip()
            temperature_data.append((date, temperature))

    # 降水量のデータを取得
    precipitation_data = []
    for row in soup.find_all('tr', class_='mtx'):
        columns = row.find_all('td')
        if len(columns) >= 13:
            date = columns[0].text.strip()
            precipitation = columns[12].text.strip()
            precipitation_data.append((date, precipitation))

    return temperature_data, precipitation_data

# 1つ目のURLからデータを取得
url1 = 'https://www.data.jma.go.jp/stats/etrn/view/daily_a1.php?prec_no=44&block_no=0371&year=2023&month=12&day=&view=p1'
temperature_data1, precipitation_data1 = scrape_weather_data(url1)

# 2つ目のURLからデータを取得
url2 = 'https://www.data.jma.go.jp/stats/etrn/view/daily_a1.php?prec_no=44&block_no=0371&year=2024&month=01&day=&view=p1'
temperature_data2, precipitation_data2 = scrape_weather_data(url2)

# 取得したデータを出力
print("1つ目のURLの気温データ:")
for date, temperature in temperature_data1:
    print(f"{date}: {temperature}℃")

print("\n1つ目のURLの降水量データ:")
for date, precipitation in precipitation_data1:
    print(f"{date}: {precipitation}mm")

print("\n2つ目のURLの気温データ:")
for date, temperature in temperature_data2:
    print(f"{date}: {temperature}℃")

print("\n2つ目のURLの降水量データ:")
for date, precipitation in precipitation_data2:
    print(f"{date}: {precipitation}mm")


/Users/yanoyuutarou/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


1つ目のURLの気温データ:
1: 0.0℃
2: 0.0℃
3: 0.0℃
4: 0.0℃
5: 0.0℃
6: 0.5℃
7: 0.0℃
8: 0.0℃
9: 0.0℃
10: 0.0℃
11: 0.5℃
12: 4.5℃
13: 2.0℃
14: 0.0℃
15: 0.0℃
16: 0.0℃
17: 0.0℃
18: 0.0℃
19: 0.0℃
20: 0.0℃
21: 0.0℃
22: 0.0℃
23: 0.0℃
24: 0.0℃
25: 0.0℃
26: 0.0℃
27: 0.0℃
28: 0.0℃
29: 0.0℃
30: 0.0℃
31: 1.0℃

1つ目のURLの降水量データ:
1: 10.8mm
2: 7.2mm
3: 13.4 )mm
4: 10.8mm
5: 8.2mm
6: 9.3mm
7: 20.1mm
8: 12.3mm
9: 6.2 )mm
10: 6.7mm
11: 9.3mm
12: 10.8mm
13: 11.8mm
14: 6.7mm
15: 24.2 )mm
16: 24.2mm
17: 19.0 )mm
18: 11.8mm
19: 7.2mm
20: 7.2 )mm
21: 15.9 )mm
22: 14.4 )mm
23: 7.2mm
24: 6.2mm
25: 8.7mm
26: 11.3mm
27: 12.9mm
28: 10.8mm
29: 6.7mm
30: 7.7mm
31: 9.3mm

2つ目のURLの気温データ:
1: 0.0℃
2: 0.0℃
3: 0.0℃
4: 0.0℃
5: 0.0℃
6: 0.0℃
7: 0.0℃
8: 0.0℃
9: 0.0℃
10: ///℃
11: ///℃
12: ///℃
13: ///℃
14: ///℃
15: ///℃
16: ///℃
17: ///℃
18: ///℃
19: ///℃
20: ///℃
21: ///℃
22: ///℃
23: ///℃
24: ///℃
25: ///℃
26: ///℃
27: ///℃
28: ///℃
29: ///℃
30: ///℃
31: ///℃

2つ目のURLの降水量データ:
1: 21.1mm
2: 6.7mm
3: 4.6mm
4: 16.5mm
5: 8.7mm
6: 9.8mm
7: 15.9m